In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

In [42]:
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
items_df = pd.read_json('items.json')
packages_df = pd.read_json('packages.json')

In [3]:
print('Items Dataframe Info:\n')
print(items_df.info())
print('Items Dataframe head:\n')
print(items_df.head())

Items Dataframe Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    60 non-null     object
 1   price   60 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB
None
Items Dataframe head:

  name  price
0   A1     98
1   A2    108
2   A3     82
3   A4     34
4   A5     52


In [29]:
unique_items = np.sort(items_df['name'].unique().flatten())
print('Total number of items: {}'.format(unique_items.size))
print('Item names: \n{}'.format(unique_items))

Total number of items: 60
Item names: 
['A1' 'A10' 'A11' 'A12' 'A13' 'A14' 'A15' 'A16' 'A17' 'A18' 'A19' 'A2'
 'A20' 'A21' 'A22' 'A23' 'A24' 'A25' 'A26' 'A27' 'A28' 'A29' 'A3' 'A30'
 'A31' 'A32' 'A33' 'A34' 'A35' 'A36' 'A37' 'A38' 'A39' 'A4' 'A40' 'A41'
 'A42' 'A43' 'A44' 'A45' 'A46' 'A47' 'A48' 'A49' 'A5' 'A50' 'A51' 'A52'
 'A53' 'A54' 'A55' 'A56' 'A57' 'A58' 'A59' 'A6' 'A60' 'A7' 'A8' 'A9']


In [9]:
print('Packages Dataframe Info:\n')
print(packages_df.info())
print('Packages Dataframe head:\n')
print(packages_df.head())

Packages Dataframe Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   total_volume  1000 non-null   float64
 1   items         1000 non-null   object 
dtypes: float64(1), object(1)
memory usage: 15.8+ KB
None
Packages Dataframe head:

   total_volume                                    items
0         36.04                      [A28, A3, A33, A59]
1        123.78  [A51, A23, A57, A5, A33, A28, A12, A47]
2         58.54                          [A10, A19, A22]
3         79.33           [A38, A28, A57, A27, A46, A45]
4         54.36                          [A40, A24, A57]


In [30]:
unique_items_included_in_packages = np.sort(packages_df['items'].explode().unique().flatten())
print('Total number of unique items included in packages: {}'.format(unique_items_included_in_packages.size))
print('Item names included in packages: \n{}'.format(unique_items_included_in_packages))

Total number of unique items included in packages: 60
Item names included in packages: 
['A1' 'A10' 'A11' 'A12' 'A13' 'A14' 'A15' 'A16' 'A17' 'A18' 'A19' 'A2'
 'A20' 'A21' 'A22' 'A23' 'A24' 'A25' 'A26' 'A27' 'A28' 'A29' 'A3' 'A30'
 'A31' 'A32' 'A33' 'A34' 'A35' 'A36' 'A37' 'A38' 'A39' 'A4' 'A40' 'A41'
 'A42' 'A43' 'A44' 'A45' 'A46' 'A47' 'A48' 'A49' 'A5' 'A50' 'A51' 'A52'
 'A53' 'A54' 'A55' 'A56' 'A57' 'A58' 'A59' 'A6' 'A60' 'A7' 'A8' 'A9']


In [21]:
items_not_included_in_packages = list(set(unique_items)-set(unique_items_included_in_packages))
num_items_not_included_in_packages = len(items_not_included_in_packages)

print('Items not included in packages: \n{}'.format(items_not_included_in_packages))
print('Number of items not included in packages: \n{}'.format(num_items_not_included_in_packages))

Items not included in packages: 
[]
Number of items not included in packages: 
0


In [40]:
item_index_map = {item_name: i_ for i_, item_name in enumerate(unique_items)}

X = np.zeros([len(packages_df),unique_items.size])
y = packages_df['total_volume'].values

for package_i_, package_items in enumerate(packages_df['items']):
    for item_name in package_items:
        if item_name in item_index_map:
            X[package_i_, item_index_map[item_name]] = 1

In [43]:
linear_model = LinearRegression(positive=True, fit_intercept=False)
linear_model.fit(X,y)

/Volumes/T7/Projects/Zalando/venv/lib/python3.9/site-packages/scipy/optimize/_nnls.py:105: RuntimeWarning: overflow encountered in matmul
  AtA = A.T @ A
/Volumes/T7/Projects/Zalando/venv/lib/python3.9/site-packages/scipy/optimize/_nnls.py:106: RuntimeWarning: overflow encountered in matmul
  Atb = b @ A  # Result is 1D - let NumPy figure it out
/Volumes/T7/Projects/Zalando/venv/lib/python3.9/site-packages/scipy/optimize/_nnls.py:155: RuntimeWarning: overflow encountered in matmul
  w[:] = Atb - AtA @ x
/Volumes/T7/Projects/Zalando/venv/lib/python3.9/site-packages/scipy/optimize/_nnls.py:155: RuntimeWarning: overflow encountered in matmul
  w[:] = Atb - AtA @ x
/Volumes/T7/Projects/Zalando/venv/lib/python3.9/site-packages/scipy/optimize/_nnls.py:155: RuntimeWarning: overflow encountered in matmul
  w[:] = Atb - AtA @ x
/Volumes/T7/Projects/Zalando/venv/lib/python3.9/site-packages/scipy/optimize/_nnls.py:155: RuntimeWarning: overflow encountered in matmul
  w[:] = Atb - AtA @ x
/Volumes

LinearRegression(fit_intercept=False, positive=True)

In [46]:
estimated_volumes = linear_model.coef_
item_volume_df = pd.DataFrame({'name': unique_items, 'volume': estimated_volumes})
item_price_volume_df = pd.merge(items_df, item_volume_df, on='name')

In [47]:
print('Item Price Volume Dataframe Info:\n')
print(item_price_volume_df.info())
print('Item Price Volume Dataframe head:\n')
print(item_price_volume_df.head())

Item Price Volume Dataframe Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   name    60 non-null     object 
 1   price   60 non-null     int64  
 2   volume  60 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.5+ KB
None
Item Price Volume Dataframe head:

  name  price     volume
0   A1     98  22.021119
1   A2    108  14.457811
2   A3     82   8.454917
3   A4     34  23.950559
4   A5     52  16.608708
